In [ ]:
!pip install transformers datasets torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


**Bert**

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, GPT2Tokenizer, GPT2LMHeadModel, LlamaTokenizer, LlamaForSequenceClassification
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [ ]:
import pandas as pd
url = "samsum-test.csv"
dataset = pd.read_csv(url)
dataset.head()


,id,dialogue,summary
0,13862856,"Hannah: Hey, do you have Betty's number?\nAman...",Hannah needs Betty's number but Amanda doesn't...
1,13729565,Eric: MACHINE!\r\nRob: That's so gr8!\r\nEric:...,Eric and Rob are going to watch a stand-up on ...
2,13680171,"Lenny: Babe, can you help me with something?\r...",Lenny can't decide which trousers to buy. Bob ...
3,13729438,"Will: hey babe, what do you want for dinner to...",Emma will be home soon and she will let Will k...
4,13828600,"Ollie: Hi , are you in Warsaw\r\nJane: yes, ju...",Jane is in Warsaw. Ollie and Jane has a party....


In [ ]:
dataset.head()
dataset.columns

Index(['id', 'dialogue', 'summary'], dtype='object')

In [ ]:
dataset.isnull().sum()


,0
id,0
dialogue,0
summary,0


In [ ]:
from sklearn.model_selection import train_test_split
train_data, val_data = train_test_split(dataset, test_size=0.2, random_state=42)
print(f"Training data: {len(train_data)} samples")
print(f"Validation data: {len(val_data)} samples")


Training data: 655 samples
Validation data: 164 samples


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("ntkhoi/BERT-Text-Summarization")
model = AutoModelForSeq2SeqLM.from_pretrained("ntkhoi/BERT-Text-Summarization")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.92k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/247M [00:00<?, ?B/s]

Config of the encoder: <class 'transformers.models.bert.modeling_bert.BertModel'> is overwritten by shared encoder config: BertConfig {
  "_name_or_path": "google/bert_uncased_L-4_H-512_A-8",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 512,
  "initializer_range": 0.02,
  "intermediate_size": 2048,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 8,
  "num_hidden_layers": 4,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "return_dict": false,
  "transformers_version": "4.46.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

Config of the decoder: <class 'transformers.models.bert.modeling_bert.BertLMHeadModel'> is overwritten by shared decoder config: BertConfig {
  "_name_or_path": "google/bert_uncased_L-4_H-512_A-8",
  "add_cross_attention": true,
  "attenti

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("ntkhoi/BERT-Text-Summarization")


def tokenize_data(examples):

    inputs = tokenizer(examples['dialogue'], padding='max_length', truncation=True, max_length=512)

    targets = tokenizer(examples['summary'], padding='max_length', truncation=True, max_length=150)


    inputs['labels'] = targets['input_ids']

    return inputs


train_data = train_data.apply(lambda row: tokenize_data(row), axis=1)
val_data = val_data.apply(lambda row: tokenize_data(row), axis=1)


In [ ]:
import torch

def convert_to_tensors(examples):
    input_ids = torch.tensor(examples['input_ids'])
    attention_mask = torch.tensor(examples['attention_mask'])
    labels = torch.tensor(examples['labels'])
    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}


train_data = list(map(lambda row: convert_to_tensors(row), train_data))
val_data = list(map(lambda row: convert_to_tensors(row), val_data))


print(train_data[0])
print(val_data[0])


{'input_ids': tensor([ 101, 3419, 1024, 2040, 1005, 1055, 2105, 1029, 2990, 1024, 2115, 5980,
         999, 3419, 1024, 2175, 6616, 4426, 2990, 2728, 1024, 1045, 1005, 1049,
        2025, 1999, 2237, 1010, 3374, 6775,  999,  102,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0, 

In [ ]:
from transformers import Trainer, TrainingArguments


training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:

trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:629: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  decoder_attention_mask = decoder_input_ids.new_tensor(decoder_input_ids != self.config.pad_token_id)
/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:649: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


Epoch,Training Loss,Validation Loss
1,0.484800,0.423579
2,0.354800,0.413012
3,0.299600,0.413855


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:629: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  decoder_attention_mask = decoder_input_ids.new_tensor(decoder_input_ids != self.config.pad_token_id)
/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:649: FutureWarning: Version v4.12.0 introduce

TrainOutput(global_step=246, training_loss=0.6286164110269004, metrics={'train_runtime': 95.6272, 'train_samples_per_second': 20.549, 'train_steps_per_second': 2.572, 'total_flos': 181001257113600.0, 'train_loss': 0.6286164110269004, 'epoch': 3.0})

In [ ]:

eval_results = trainer.evaluate()
print(eval_results)


{'eval_loss': 0.4138546884059906, 'eval_runtime': 1.5462, 'eval_samples_per_second': 106.069, 'eval_steps_per_second': 13.582, 'epoch': 3.0}


In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("ntkhoi/BERT-Text-Summarization")
model = AutoModelForSeq2SeqLM.from_pretrained("ntkhoi/BERT-Text-Summarization")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
input_text = ""
input_ids = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=512)
input_ids = {key: value.to(device) for key, value in input_ids.items()}
generated_ids = model.generate(input_ids['input_ids'], attention_mask=input_ids['attention_mask'], max_length=150)
summary = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print('summary')

Config of the encoder: <class 'transformers.models.bert.modeling_bert.BertModel'> is overwritten by shared encoder config: BertConfig {
  "_name_or_path": "google/bert_uncased_L-4_H-512_A-8",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 512,
  "initializer_range": 0.02,
  "intermediate_size": 2048,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 8,
  "num_hidden_layers": 4,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "return_dict": false,
  "transformers_version": "4.46.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

Config of the decoder: <class 'transformers.models.bert.modeling_bert.BertLMHeadModel'> is overwritten by shared decoder config: BertConfig {
  "_name_or_path": "google/bert_uncased_L-4_H-512_A-8",
  "add_cross_attention": true,
  "attenti

summary


**GPT-2**

In [ ]:
# Initialize the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Check if the tokenizer has a pad_token, if not, use eos_token as pad_token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # Set padding token to eos token


In [ ]:
# Preprocessing function
def preprocess_data(examples):
    # Tokenize inputs and targets
    inputs = tokenizer(
        examples['dialogue'],
        max_length=512,
        truncation=True,
        padding="max_length"
    )
    targets = tokenizer(
        examples['summary'],
        max_length=128,
        truncation=True,
        padding="max_length"
    )
    # Add labels for supervised learning
    inputs['labels'] = targets['input_ids']
    return inputs


In [ ]:
# Apply preprocessing to the entire dataset
tokenized_datasets = Dataset.from_pandas(dataset).map(preprocess_data, batched=True)
tokenized_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

In [ ]:
# Split data into training and validation sets (80/20 split)
train_test_split = tokenized_datasets.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
val_dataset = train_test_split['test']


In [ ]:
# Initialize GPT-2 model
model = GPT2LMHeadModel.from_pretrained("gpt2")


In [ ]:
# Data collator for padding and formatting
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Set to False for causal language modeling
)


In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-summarization",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    weight_decay=0.01,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    save_total_limit=2,
    report_to="none"  # Avoid sending logs to external platforms
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

# Fine-tune the model
trainer.train()


<ipython-input-18-167dbf09553a>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,2.991400,2.827097
2,2.672700,2.808397
3,2.509000,2.810966


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=492, training_loss=2.811428508138269, metrics={'train_runtime': 455.5228, 'train_samples_per_second': 4.314, 'train_steps_per_second': 1.08, 'total_flos': 513438842880000.0, 'train_loss': 2.811428508138269, 'epoch': 3.0})

In [ ]:
# Evaluate the fine-tuned model
trainer.evaluate()


{'eval_loss': 2.8083972930908203,
 'eval_runtime': 7.0296,
 'eval_samples_per_second': 23.33,
 'eval_steps_per_second': 5.832,
 'epoch': 3.0}

In [ ]:
# Save the fine-tuned model and tokenizer
model.save_pretrained("./gpt2-summarization")
tokenizer.save_pretrained("./gpt2-summarization")


('./gpt2-summarization/tokenizer_config.json',
 './gpt2-summarization/special_tokens_map.json',
 './gpt2-summarization/vocab.json',
 './gpt2-summarization/merges.txt',
 './gpt2-summarization/added_tokens.json')

In [ ]:

eval_results = trainer.evaluate()
print(eval_results)


{'eval_loss': 2.8083972930908203, 'eval_runtime': 7.0743, 'eval_samples_per_second': 23.183, 'eval_steps_per_second': 5.796, 'epoch': 3.0}


In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Set pad_token to eos_token (since GPT-2 doesn't have a pad token)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Move the model to the GPU (if available) or CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Input text for summarization
input_text = "Your input text goes here. This could be a long article or any content you want to summarize."

# Tokenize the input text
input_ids = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=512)

# Move input tensors to the correct device (GPU/CPU)
input_ids = {key: value.to(device) for key, value in input_ids.items()}

# Generate summary using the model
generated_ids = model.generate(
    input_ids['input_ids'],
    attention_mask=input_ids['attention_mask'],
    max_length=150,          # Limit the summary length to 150 tokens
    num_beams=4,            # Use beam search to improve output quality
    temperature=0.7,        # Control randomness in the output
    top_k=50,               # Limit the next token options
    top_p=0.95,             # Use nucleus sampling
    early_stopping=True     # Stop when the model is confident the text is complete
)

# Decode the generated ids to get the summary text
summary = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

# Print the generated summary
print("Summary:", summary)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Summary: Your input text goes here. This could be a long article or any content you want to summarize.

If you have any questions or comments, please feel free to contact me.


**LLAMA**

In [ ]:
!pip install transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import Dataset


In [ ]:
from transformers import (
    LlamaTokenizer, LlamaForCausalLM, Trainer, TrainingArguments
)
import pandas as pd

In [ ]:
def preprocess_data(data, tokenizer, max_length):
    # Replace 'dialogue' with the correct column name from your dataset
    return tokenizer(data["dialogue"], padding="max_length", truncation=True, max_length=max_length)
